In [1]:
t = (0,1,2)

In [3]:
result = []
division_tree = []
division_modularities = []

In [23]:
return_tuple = result, division_tree
# if division_tree:
#     return result, division_tree
# if return_modularities:
#     return result, division_modularities

In [24]:
return_tuple

([], [])

In [25]:
return_tuple = (*return_tuple, division_modularities)
return_tuple

([], [], [])